# __From raw twitter scrape json to clean fine tune babyfood jsonl__

## snatch tweets from json
reads a file that is a list of dictionaries where the target (tweet) is the value with a key named "text"

In [62]:
import json
 
# Opening JSON file
f = open('')
json_file = json.load(f)

text_file = []

for dictionary in json_file:
    text_file.append(dictionary["text"])
    

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## tweet cleaner
if we decide hashtags, emojis or @mentions ae a part of contemporary expressions that belong into poetry we could comment out

In [63]:
import re
import emoji


def tweet_cleaner(tweet):
    # remove mentions
    mentions_free = re.sub("@\w+", "", tweet)
    # remove hashtags
    hashtag_free = re.sub("#\w+", "", mentions_free)
    # special shortcuts
    rt_free = re.sub("^RT", "", hashtag_free)
    # remove url 
    url_free = re.sub(r'http\S+', '', rt_free)
    # remove emojis
    emo_free = emoji.get_emoji_regexp().sub(u'', url_free)
    # remove white space
    stripped = emo_free.strip()
  
    return stripped

In [64]:
clean_tweets = [tweet_cleaner(x)for x in text_file]
len(clean_tweets)

0

## unicode
this removes stuff like \xa \n and so on

In [65]:
def unicode_maker(text):
    # creating a unicode string
    text_encode = text.encode(encoding="ascii", errors="ignore")
    text_decode = text_encode.decode()
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

uni_tweets = [unicode_maker(x)for x in clean_tweets]
uni_tweets

[]

## remove punctuation, numbers etc.

In [66]:
import string

In [67]:
def number_punct(sentence):
    # remove numbers
    number_free = ''.join(word for word in sentence if not word.isdigit())
    # remove punctuation
    punctuation_free = "".join(
        [i for i in number_free if i not in string.punctuation])
    # lower case
    upper_free = punctuation_free.lower()
    
    return upper_free

number_free = [number_punct(x) for x in uni_tweets]
number_free

[]

In [68]:
## removes tweets with less than two words
more_than_two = [x for x in number_free if (len(x.split())>2)]

## removes duplicates
clean_set = set(more_than_two)
clean_set

set()

## remove empty strings and excess whitespaces
strings become empty when they where just made of mentions, url, hashtags etc.

In [69]:
# remove empty strings
no_empty_strings = [i for i in clean_set if i]
# remove outer whitespace 
stripped = [x.strip() for x in no_empty_strings]
# remove inner whitespace 
one_space =  [re.sub(' +', ' ', x) for x in stripped]
one_space

[]

## add whitespace in the beginning for openai's 
tokenization and stopping . END

In [70]:
start_end = [f" {x}. END" for x in stripped] 
start_end

[]

## Creates a DataFrame with a prompt and completion column

In [71]:
import pandas as pd

In [25]:
# keyword = json_file[0]["segment"]
df_prompt_comp = pd.DataFrame({"prompt": "", "completion":start_end})
df_dub_free = df_prompt_comp.drop_duplicates()

__saves json babyfood__

In [26]:
df_dub_free.to_json("musician1.json", orient = "records", lines= True)

# __from song parts to babyfood__
saves some song examples in a txt file and get json with linebreaks and end bit (END) starting whitespace - songparts are devided by blank line (\n\n respectively) 

In [72]:
import re

In [73]:
# load txt file with song examples - examples are divided by whiteline
rap_file = list(open("empty.txt", "r"))
rap_list = "".join(x for x in rap_file)

# split examples by whiteline \n\n
chunks = re.split('\n\n', rap_list)

# replace \' by '
apo = [sub.replace("\'", "'") for sub in chunks]

# remove empty strings
no_empty_rap = [i for i in apo if i]

# put leading whitespace and END bit for better tokenization and GTP3 learning
space_rap = [f" {x} END" for x in no_empty_rap] 

# remove unicode
string_decode =  [x.encode("ascii", "ignore").decode() for x in space_rap] 

# make a pandas Dataframe
df_rap = pd.DataFrame({"prompt": "...->", "completion":string_decode})

# save into a GTP3 learnable jsonl with linebreaks and END bit to give as a example document 
df_rap.to_json("rap_doc5.json", orient = "records", lines= True)

# Not Used

creates text file without prompts just clean tweets line by line

In [44]:
one_long_string = ".\n".join(one_space)

with open("file.txt", "w") as output:
    output.write(one_long_string)

FileNotFoundError: [Errno 2] No such file or directory: ''

## Tokenize
spits out a list of lists of important words 

In [45]:
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))


def text_to_token(sentence):
    # tokenize
    word_tokens = word_tokenize(sentence) 
    # remove stopwords
    important_words = [w for w in word_tokens if not w in stop_words] 
    
    return important_words

In [49]:
tokens = [text_to_token(s) for s in one_space]
tokens

[['donde', 'esta', 'juan', 'pablo'],
 ['love',
  'especially',
  'note',
  'women',
  'work',
  'care',
  'deeply',
  'bring',
  'lived',
  'experience',
  'work',
  'f'],
 ['love',
  'bullshit',
  'life',
  'cut',
  'cant',
  'believe',
  'much',
  'accumulated',
  'hate',
  'think',
  'stuff',
  'usually',
  'stay',
  'busy',
  'think'],
 ['congratulations',
  'welldeserved',
  'recognition',
  'like',
  'im',
  'optimistic',
  'able',
  'bring',
  'pandemic',
  'end',
  'work',
  'many',
  'others',
  'also',
  'help',
  'us',
  'better',
  'prepare',
  'future',
  'pandemics'],
 ['four',
  'ways',
  'america',
  'advance',
  'leadership',
  'climate',
  'change',
  'year',
  'put',
  'world',
  'path',
  'zero',
  'emissions'],
 ['juneteenth', 'day', 'profound', 'weight', 'profound', 'power'],
 ['world',
  'inequality',
  'lab',
  'newsletter',
  'march',
  'welcome',
  'fourth',
  'world',
  'inequality',
  'lab',
  'newsletter',
  'covers',
  'main',
  'developments',
  'since',


## Lemmatize
gets rid of plurals aso. without mutliating words like stemming.

In [47]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

def get_lemm(l):
    lemm = [lemma.lemmatize(word) for word in l]
    return lemm

In [48]:
lemmas = [get_lemm(s) for s in tokens]
lemmas

[['donde', 'esta', 'juan', 'pablo'],
 ['love',
  'especially',
  'note',
  'woman',
  'work',
  'care',
  'deeply',
  'bring',
  'lived',
  'experience',
  'work',
  'f'],
 ['love',
  'bullshit',
  'life',
  'cut',
  'cant',
  'believe',
  'much',
  'accumulated',
  'hate',
  'think',
  'stuff',
  'usually',
  'stay',
  'busy',
  'think'],
 ['congratulation',
  'welldeserved',
  'recognition',
  'like',
  'im',
  'optimistic',
  'able',
  'bring',
  'pandemic',
  'end',
  'work',
  'many',
  'others',
  'also',
  'help',
  'u',
  'better',
  'prepare',
  'future',
  'pandemic'],
 ['four',
  'way',
  'america',
  'advance',
  'leadership',
  'climate',
  'change',
  'year',
  'put',
  'world',
  'path',
  'zero',
  'emission'],
 ['juneteenth', 'day', 'profound', 'weight', 'profound', 'power'],
 ['world',
  'inequality',
  'lab',
  'newsletter',
  'march',
  'welcome',
  'fourth',
  'world',
  'inequality',
  'lab',
  'newsletter',
  'cover',
  'main',
  'development',
  'since',
  'octo